# kNN classifier results

In [ ]:
"""kNN classifier results

"""

In [ ]:
import pandas as pd  # import libraries
import numpy as np
from sklearn.neighbors import KNeighborsClassifier
from sklearn import preprocessing
from sklearn.model_selection import TimeSeriesSplit
from sklearn.metrics import classification_report
from imblearn.over_sampling import RandomOverSampler

# import data
df = pd.read_csv('data/SCADA_downtime_merged.csv', skip_blank_lines=True)

list1 = list(df['turbine_id'].unique())  # list of turbines to plot
list1 = sorted(list1, key=int)  # sort turbines in ascending order
list2 = list(df['TurbineCategory_id'].unique())  # list of categories
list2 = [g for g in list2 if g >= 0]  # remove NaN from list
list2 = sorted(list2, key=int)  # sort categories in ascending order
# categories to remove
list2 = [m for m in list2 if m not in (1, 12, 13, 14, 15, 17, 21, 22)]
list4 = list(range(0, 14))  # index for classes
list5 = list(zip(list4, list2))

for x in list1:  # filter only data for turbine x
    dfx = df[(df['turbine_id'] == x)].copy()
    for y in list2:
        # copying fault to new column (mins)
        # (fault when turbine category id is y)

        def ff(c):
            if c['TurbineCategory_id'] == y:
                return 0
            else:
                return 1

        dfx['mins'] = dfx.apply(ff, axis=1)

        # sort values by timestamp in descending order
        dfx = dfx.sort_values(by='timestamp', ascending=False)
        dfx.reset_index(drop=True, inplace=True)  # reset index

        # assigning value to first cell if it's not 0
        if dfx.loc[0, 'mins'] == 0:
            dfx.set_value(0, 'mins', 0)
        else:
            dfx.set_value(0, 'mins', 999999999)

        for i, e in enumerate(dfx['mins']):
            # using previous value's row to evaluate time
            if e == 1:
                dfx.at[i, 'mins'] = dfx.at[i - 1, 'mins'] + 10

        dfx = dfx.sort_values(by='timestamp')  # sort in ascending order
        dfx.reset_index(drop=True, inplace=True)  # reset index
        dfx['hours'] = dfx['mins'].astype(np.int64)
        # convert to hours, then round to nearest hour
        dfx['hours'] = dfx['hours'] / 60
        dfx['hours'] = round(dfx['hours']).astype(np.int64)

        def f11(c):  # > 48 hours - label as normal (9999)
            if c['hours'] > 48:
                return 9999
            else:
                return c['hours']

        dfx['hours'] = dfx.apply(f11, axis=1)

        def f22(c,):
            # filter out curtailment - curtailed when turbine is pitching
            # outside 0deg<= normal <=3.5deg
            if (0 <= c['pitch'] <= 3.5 or c['hours'] != 9999 or (
                    (c['pitch'] > 3.5 or c['pitch'] < 0) and (
                        c['ap_av'] <= (0.1 * dfx['ap_av'].max())
                        or c['ap_av'] >= (0.9 * dfx['ap_av'].max())))):
                return 'normal'
            else:
                return 'curtailed'

        dfx['curtailment'] = dfx.apply(f22, axis=1)

        def f3(c,):
            # filter unusual readings, i.e. for normal operation,
            # power <= 0 in operating wind speeds, power >100...
            # before cut-in, runtime < 600 and other downtime categories
            if c['hours'] == 9999 and ((
                        3 < c['ws_av'] < 25 and (
                            c['ap_av'] <= 0
                            or c['runtime'] < 600
                            or c['EnvironmentalCategory_id'] > 1
                            or c['GridCategory_id'] > 1
                            or c['InfrastructureCategory_id'] > 1
                            or c['AvailabilityCategory_id'] == 2
                            or 12 <= c['TurbineCategory_id'] <= 15
                            or 21 <= c['TurbineCategory_id'] <= 22))
                    or (c['ws_av'] < 3 and c['ap_av'] > 100)):
                return 'unusual'
            else:
                return 'normal'

        dfx['unusual'] = dfx.apply(f3, axis=1)

        def f4(c):  # round to 6 hour intervals
            if c['hours'] == 0:
                return 10
            elif 1 <= c['hours'] <= 6:
                return 11
            elif 7 <= c['hours'] <= 12:
                return 12
            elif 13 <= c['hours'] <= 18:
                return 13
            elif 19 <= c['hours'] <= 24:
                return 14
            elif 25 <= c['hours'] <= 30:
                return 15
            elif 31 <= c['hours'] <= 36:
                return 16
            elif 37 <= c['hours'] <= 42:
                return 17
            elif 43 <= c['hours'] <= 48:
                return 18
            else:
                return 19  # normal

        dfx['hours6'] = dfx.apply(f4, axis=1)

        def f5(c):  # change label for unusual and curtailed data (20)
            if c['unusual'] == 'unusual' or c['curtailment'] == 'curtailed':
                return 20
            else:
                return c['hours6']

        dfx['hours_%s' % y] = dfx.apply(f5, axis=1)

        dfx = dfx.drop('hours6', axis=1)  # drop unnecessary columns
        dfx = dfx.drop('hours', axis=1)
        dfx = dfx.drop('mins', axis=1)
        dfx = dfx.drop('curtailment', axis=1)
        dfx = dfx.drop('unusual', axis=1)

    # separate features from classes for classification
    features = [
        'ap_av', 'ws_av', 'wd_av', 'pitch', 'ap_max', 'ap_dev',
        'reactive_power', 'rs_av', 'gen_sp', 'nac_pos']
    classes = [col for col in dfx.columns if 'hours' in col]
    list6 = features + classes  # list of columns to copy into new df
    df2 = dfx[list6].copy()
    df2 = df2.dropna()  # drop NaNs
    X = df2[features]
    X = preprocessing.normalize(X)  # normalise features to values b/w 0 and 1
    Y = df2[classes]
    Y = Y.as_matrix()  # convert from pd dataframe to np array
    # cross validation using time series split
    tscv = TimeSeriesSplit(n_splits=5)

    knn = KNeighborsClassifier(weights='distance', n_jobs=-1)
    for (m, n) in list5:
        Ym = Y[:, m]
        for train_index, test_index in tscv.split(X):
            # looping for each cross validation fold
            # split train and test sets
            X_train, X_test = X[train_index], X[test_index]
            Y_train, Y_test = Ym[train_index], Ym[test_index]
            if len(set(Y_train)) > 1:
                ros = RandomOverSampler()
                Xt, Yt = ros.fit_sample(X_train, Y_train)
            else:
                Xt, Yt = X_train, Y_train
            knn1 = knn.fit(Xt, Yt)  # fit to classifier and predict
            Yp = knn1.predict(X_test)
            print(
                'Classification report for turbine %s, turbine category %s'
                % (x, n))
            print(classification_report(Y_test, Yp, digits=6))
        print('------------------------------------------------------------')

Classification report for turbine 1, turbine category 2.0
             precision    recall  f1-score   support

         19   0.983002  0.953712  0.968135     16311
         20   0.506536  0.742337  0.602176      1044

avg / total   0.954340  0.940997  0.946121     17355

Classification report for turbine 1, turbine category 2.0
             precision    recall  f1-score   support

         19   0.939021  0.936903  0.937961     15072
         20   0.589555  0.598336  0.593913      2283

avg / total   0.893050  0.892365  0.892702     17355

Classification report for turbine 1, turbine category 2.0
             precision    recall  f1-score   support

         19   0.649076  0.917014  0.760125      9845
         20   0.762914  0.350067  0.479920      7510

avg / total   0.698337  0.671680  0.638872     17355

Classification report for turbine 1, turbine category 2.0
             precision    recall  f1-score   support

         19   0.883899  0.958171  0.919538     13579
         20   0.

C:\Program Files\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1113: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


Classification report for turbine 1, turbine category 3.0
             precision    recall  f1-score   support

         10   0.000000  0.000000  0.000000       249
         11   0.000000  0.000000  0.000000       192
         12   0.000000  0.000000  0.000000       180
         13   0.000000  0.000000  0.000000       180
         14   0.000000  0.000000  0.000000       180
         15   0.000000  0.000000  0.000000       155
         16   0.000000  0.000000  0.000000       144
         17   0.000000  0.000000  0.000000       144
         18   0.000000  0.000000  0.000000       144
         19   0.887408  0.915288  0.901132     14260
         20   0.441176  0.667976  0.531388      1527

avg / total   0.767970  0.810833  0.787184     17355

Classification report for turbine 1, turbine category 3.0
             precision    recall  f1-score   support

         10   0.000000  0.000000  0.000000         0
         11   0.000000  0.000000  0.000000         0
         12   0.000000  0.000000

C:\Program Files\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1115: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


Classification report for turbine 1, turbine category 3.0
             precision    recall  f1-score   support

         10   0.013699  0.011905  0.012739        84
         11   0.018072  0.083333  0.029703        36
         12   0.016129  0.083333  0.027027        36
         13   0.000000  0.000000  0.000000        36
         14   0.019048  0.055556  0.028369        36
         15   0.000000  0.000000  0.000000        36
         16   0.000000  0.000000  0.000000        36
         17   0.000000  0.000000  0.000000        36
         18   0.000000  0.000000  0.000000        36
         19   0.869747  0.906191  0.887595     13293
         20   0.751868  0.518157  0.613509      3690

avg / total   0.826218  0.804782  0.810532     17355

Classification report for turbine 1, turbine category 3.0
             precision    recall  f1-score   support

         10   0.000000  0.000000  0.000000        17
         11   0.000000  0.000000  0.000000        72
         12   0.000000  0.000000

...

Classification report for turbine 3, turbine category 7.0
             precision    recall  f1-score   support

         10   0.476190  0.505618  0.490463       178
         11   0.000000  0.000000  0.000000        41
         12   0.000000  0.000000  0.000000        36
         13   0.008065  0.027778  0.012500        36
         14   0.000000  0.000000  0.000000        36
         15   0.000000  0.000000  0.000000        36
         16   0.000000  0.000000  0.000000        36
         17   0.000000  0.000000  0.000000        36
         18   0.000000  0.000000  0.000000        36
         19   0.966870  0.892767  0.928342     18306
         20   0.728684  0.836247  0.778769      2974

avg / total   0.917276  0.869891  0.891823     21751

Classification report for turbine 3, turbine category 7.0
             precision    recall  f1-score   support

         10   0.000000  0.000000  0.000000         0
         11   0.000000  0.000000  0.000000         0
         12   0.000000  0.000000

Classification report for turbine 3, turbine category 9.0
             precision    recall  f1-score   support

         10   0.078704  0.087179  0.082725       195
         11   0.007634  0.013072  0.009639       153
         12   0.000000  0.000000  0.000000       103
         13   0.000000  0.000000  0.000000        72
         14   0.004566  0.013889  0.006873        72
         15   0.005263  0.013889  0.007634        72
         16   0.000000  0.000000  0.000000        72
         17   0.006711  0.013889  0.009050        72
         18   0.009709  0.027778  0.014388        72
         19   0.880089  0.826904  0.852668     15373
         20   0.721935  0.711556  0.716708      5495

avg / total   0.805252  0.765298  0.784640     21751

Classification report for turbine 3, turbine category 9.0
             precision    recall  f1-score   support

         10   0.000000  0.000000  0.000000        14
         11   0.006757  0.018519  0.009901       108
         12   0.015528  0.046296

Classification report for turbine 3, turbine category 11.0
             precision    recall  f1-score   support

         10   0.007782  0.100000  0.014440        20
         11   0.006652  0.020833  0.010084       144
         12   0.015707  0.041667  0.022814       144
         13   0.002358  0.007692  0.003610       130
         14   0.002183  0.009259  0.003534       108
         15   0.003431  0.018519  0.005789       108
         16   0.002747  0.009259  0.004237       108
         17   0.006557  0.018519  0.009685       108
         18   0.004975  0.018519  0.007843       108
         19   0.940669  0.782705  0.854447     18514
         20   0.662132  0.797255  0.723438      2259

avg / total   0.869713  0.749943  0.802829     21751

------------------------------------------------------------------------
Classification report for turbine 3, turbine category 16.0
             precision    recall  f1-score   support

         19   0.959149  0.953828  0.956481     17305
         2

Classification report for turbine 3, turbine category 19.0
             precision    recall  f1-score   support

         19   0.986285  0.954078  0.969914     19446
         20   0.696259  0.888069  0.780553      2305

avg / total   0.955550  0.947083  0.949847     21751

------------------------------------------------------------------------
Classification report for turbine 3, turbine category 20.0
             precision    recall  f1-score   support

         19   0.959841  0.948859  0.954318     17305
         20   0.809432  0.845479  0.827063      4446

avg / total   0.929097  0.927727  0.928307     21751

Classification report for turbine 3, turbine category 20.0
             precision    recall  f1-score   support

         19   0.980050  0.942245  0.960776     18665
         20   0.716763  0.883992  0.791642      3086

avg / total   0.942695  0.933980  0.936779     21751

Classification report for turbine 3, turbine category 20.0
             precision    recall  f1-score   s

Classification report for turbine 4, turbine category 3.0
             precision    recall  f1-score   support

         10   0.000000  0.000000  0.000000         0
         11   0.000000  0.000000  0.000000         0
         12   0.000000  0.000000  0.000000         0
         13   0.000000  0.000000  0.000000         0
         14   0.000000  0.000000  0.000000         0
         15   0.000000  0.000000  0.000000         0
         16   0.000000  0.000000  0.000000         0
         17   0.000000  0.000000  0.000000         0
         18   0.000000  0.000000  0.000000         0
         19   0.987934  0.790079  0.877998     20622
         20   0.213364  0.810671  0.337816      1162

avg / total   0.946617  0.791177  0.849183     21784



Classification report for turbine 4, turbine category 2.0
             precision    recall  f1-score   support

         10   0.041494  0.200000  0.068729        50
         11   0.018100  0.222222  0.033473        36
         12   0.000000  0.000000  0.000000        36
         13   0.002896  0.083333  0.005597        36
         14   0.005599  0.138889  0.010764        36
         15   0.000000  0.000000  0.000000        36
         16   0.000000  0.000000  0.000000        36
         17   0.000000  0.000000  0.000000        36
         18   0.000000  0.000000  0.000000        36
         19   0.952504  0.623244  0.753474     17794
         20   0.756137  0.767525  0.761788      3652

avg / total   0.904944  0.638955  0.743417     21784

Classification report for turbine 4, turbine category 2.0
             precision    recall  f1-score   support

         10   0.010582  0.148148  0.019753        27
         11   0.016000  0.024845  0.019465       161
         12   0.000000  0.000000

C:\Program Files\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1115: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


             precision    recall  f1-score   support

         10   0.000000  0.000000  0.000000         0
         11   0.000000  0.000000  0.000000         0
         12   0.000000  0.000000  0.000000         0
         13   0.000000  0.000000  0.000000         0
         14   0.000000  0.000000  0.000000         0
         15   0.000000  0.000000  0.000000         0
         16   0.000000  0.000000  0.000000         0
         17   0.000000  0.000000  0.000000         0
         18   0.000000  0.000000  0.000000         0
         19   0.986854  0.735331  0.842725     20622
         20   0.204255  0.784854  0.324151      1162

avg / total   0.945109  0.737973  0.815064     21784

Classification report for turbine 4, turbine category 2.0
             precision    recall  f1-score   support

         10   0.000000  0.000000  0.000000         0
         11   0.000000  0.000000  0.000000         0
         12   0.000000  0.000000  0.000000         0
         13   0.000000  0.000000  0.0

C:\Program Files\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1113: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


Classification report for turbine 4, turbine category 3.0
             precision    recall  f1-score   support

         10   0.000000  0.000000  0.000000         0
         11   0.000000  0.000000  0.000000         0
         12   0.000000  0.000000  0.000000         0
         13   0.000000  0.000000  0.000000         0
         14   0.000000  0.000000  0.000000         0
         15   0.000000  0.000000  0.000000         0
         16   0.000000  0.000000  0.000000         0
         17   0.000000  0.000000  0.000000         0
         18   0.000000  0.000000  0.000000         0
         19   0.958001  0.817821  0.882378     15117
         20   0.808757  0.881056  0.843360      6667

avg / total   0.912325  0.837174  0.870437     21784

Classification report for turbine 4, turbine category 3.0
             precision    recall  f1-score   support

         10   0.000000  0.000000  0.000000         4
         11   0.051020  0.138889  0.074627        36
         12   0.181818  0.055556

...

Classification report for turbine 8, turbine category 10.0
             precision    recall  f1-score   support

         10   0.075180  0.501845  0.130769       271
         11   0.010135  0.016667  0.012605       180
         12   0.000000  0.000000  0.000000       117
         13   0.000000  0.000000  0.000000       108
         14   0.009615  0.009259  0.009434       108
         15   0.027027  0.037037  0.031250       108
         16   0.008351  0.074074  0.015009       108
         17   0.053097  0.055556  0.054299       108
         18   0.000000  0.000000  0.000000       108
         19   0.881541  0.827209  0.853511     11893
         20   0.640619  0.370470  0.469454      4470

avg / total   0.761165  0.662836  0.699634     17579

Classification report for turbine 8, turbine category 10.0
             precision    recall  f1-score   support

         10   0.009626  0.642857  0.018967        14
         11   0.000000  0.000000  0.000000        72
         12   0.000000  0.0000

Classification report for turbine 8, turbine category 18.0
             precision    recall  f1-score   support

         10   0.000000  0.000000  0.000000        30
         11   0.000000  0.000000  0.000000       180
         12   0.000000  0.000000  0.000000       180
         13   0.000000  0.000000  0.000000       180
         14   0.000000  0.000000  0.000000       180
         15   0.000000  0.000000  0.000000       180
         16   0.000000  0.000000  0.000000       180
         17   0.000000  0.000000  0.000000       180
         18   0.000000  0.000000  0.000000       180
         19   0.773642  0.925570  0.842814     12710
         20   0.562158  0.392468  0.462231      3399

avg / total   0.668056  0.745094  0.698748     17579

Classification report for turbine 8, turbine category 18.0
             precision    recall  f1-score   support

         10   0.000000  0.000000  0.000000        24
         11   0.000000  0.000000  0.000000       144
         12   0.041176  0.0972

Classification report for turbine 9, turbine category 2.0
             precision    recall  f1-score   support

         10   0.044444  0.048077  0.046189       208
         11   0.016484  0.016393  0.016438       183
         12   0.000000  0.000000  0.000000       125
         13   0.000000  0.000000  0.000000        58
         14   0.000000  0.000000  0.000000        36
         15   0.000000  0.000000  0.000000        26
         16   0.000000  0.000000  0.000000         4
         17   0.000000  0.000000  0.000000        36
         18   0.000000  0.000000  0.000000        36
         19   0.903371  0.919082  0.911159     16214
         20   0.828647  0.676296  0.744760      3704

avg / total   0.859371  0.844401  0.850448     20630

Classification report for turbine 9, turbine category 2.0
             precision    recall  f1-score   support

         10   0.026087  0.050000  0.034286        60
         11   0.000000  0.000000  0.000000        36
         12   0.000000  0.000000

Classification report for turbine 9, turbine category 5.0
             precision    recall  f1-score   support

         10   0.023810  0.166667  0.041667         6
         11   0.000000  0.000000  0.000000        36
         12   0.000000  0.000000  0.000000        36
         13   0.000000  0.000000  0.000000        36
         14   0.000000  0.000000  0.000000        36
         15   0.000000  0.000000  0.000000        36
         16   0.000000  0.000000  0.000000        36
         17   0.000000  0.000000  0.000000        36
         18   0.000000  0.000000  0.000000        36
         19   0.923206  0.926369  0.924785     16637
         20   0.809642  0.708300  0.755588      3699

avg / total   0.889694  0.874115  0.881280     20630

Classification report for turbine 9, turbine category 5.0
             precision    recall  f1-score   support

         10   0.000000  0.000000  0.000000         0
         11   0.000000  0.000000  0.000000         0
         12   0.000000  0.000000

Classification report for turbine 9, turbine category 8.0
             precision    recall  f1-score   support

         10   0.320225  0.633333  0.425373       360
         11   0.000000  0.000000  0.000000       108
         12   0.000000  0.000000  0.000000       108
         13   0.000000  0.000000  0.000000       108
         14   0.000000  0.000000  0.000000       108
         15   0.004673  0.009259  0.006211       108
         16   0.000000  0.000000  0.000000       108
         17   0.000000  0.000000  0.000000       108
         18   0.000000  0.000000  0.000000       108
         19   0.874786  0.804559  0.838204     10177
         20   0.852601  0.903782  0.877446      9229

avg / total   0.818572  0.812312  0.813483     20630

Classification report for turbine 9, turbine category 8.0
             precision    recall  f1-score   support

         10   0.000000  0.000000  0.000000        68
         11   0.000000  0.000000  0.000000        72
         12   0.000000  0.000000

Classification report for turbine 9, turbine category 2.0
             precision    recall  f1-score   support

         19   0.930997  0.972653  0.951369     17589
         20   0.786602  0.583032  0.669688      3041

avg / total   0.909712  0.915221  0.909848     20630

Classification report for turbine 9, turbine category 2.0
             precision    recall  f1-score   support

         10   0.000000  0.000000  0.000000        79
         11   0.000000  0.000000  0.000000        55
         12   0.000000  0.000000  0.000000        36
         13   0.000000  0.000000  0.000000        36
         14   0.000000  0.000000  0.000000        36
         15   0.000000  0.000000  0.000000        46
         16   0.000000  0.000000  0.000000        72
         17   0.000000  0.000000  0.000000        72
         18   0.000000  0.000000  0.000000        72
         19   0.846823  0.889872  0.867814     10288
         20   0.880130  0.878431  0.879280      9838

avg / total   0.842018  0.86267

C:\Program Files\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1113: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


Classification report for turbine 9, turbine category 2.0
             precision    recall  f1-score   support

         10   0.044444  0.048077  0.046189       208
         11   0.016484  0.016393  0.016438       183
         12   0.000000  0.000000  0.000000       125
         13   0.000000  0.000000  0.000000        58
         14   0.000000  0.000000  0.000000        36
         15   0.000000  0.000000  0.000000        26
         16   0.000000  0.000000  0.000000         4
         17   0.000000  0.000000  0.000000        36
         18   0.000000  0.000000  0.000000        36
         19   0.909130  0.913223  0.911172     16214
         20   0.807240  0.704374  0.752307      3704

avg / total   0.860054  0.844838  0.851813     20630

Classification report for turbine 9, turbine category 2.0
             precision    recall  f1-score   support

         10   0.025641  0.050000  0.033898        60
         11   0.000000  0.000000  0.000000        36
         12   0.000000  0.000000

C:\Program Files\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1115: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


Classification report for turbine 9, turbine category 2.0
             precision    recall  f1-score   support

         10   0.110345  0.307692  0.162437        52
         11   0.000000  0.000000  0.000000        36
         12   0.000000  0.000000  0.000000        36
         13   0.000000  0.000000  0.000000        36
         14   0.000000  0.000000  0.000000        36
         15   0.000000  0.000000  0.000000        36
         16   0.000000  0.000000  0.000000        36
         17   0.000000  0.000000  0.000000        36
         18   0.000000  0.000000  0.000000        36
         19   0.966232  0.938027  0.951921     17540
         20   0.790441  0.860000  0.823755      2750

avg / total   0.927153  0.912942  0.919557     20630

------------------------------------------------------------------------
Classification report for turbine 9, turbine category 3.0
             precision    recall  f1-score   support

         19   0.933806  0.972881  0.952943     17589
         20 

...

Classification report for turbine 25, turbine category 5.0
             precision    recall  f1-score   support

         10   0.000000  0.000000  0.000000         0
         11   0.000000  0.000000  0.000000         0
         12   0.000000  0.000000  0.000000         0
         13   0.000000  0.000000  0.000000         0
         14   0.000000  0.000000  0.000000         0
         15   0.000000  0.000000  0.000000         0
         16   0.000000  0.000000  0.000000         0
         17   0.000000  0.000000  0.000000         0
         18   0.000000  0.000000  0.000000         0
         19   0.987364  0.856679  0.917391     16327
         20   0.336496  0.741697  0.462956       813

avg / total   0.956492  0.851225  0.895836     17140

Classification report for turbine 25, turbine category 5.0
             precision    recall  f1-score   support

         10   0.000000  0.000000  0.000000        92
         11   0.000000  0.000000  0.000000        96
         12   0.000000  0.0000

Classification report for turbine 25, turbine category 7.0
             precision    recall  f1-score   support

         10   0.000000  0.000000  0.000000         0
         11   0.000000  0.000000  0.000000         0
         12   0.000000  0.000000  0.000000         0
         13   0.000000  0.000000  0.000000         0
         14   0.000000  0.000000  0.000000         0
         15   0.000000  0.000000  0.000000         0
         16   0.000000  0.000000  0.000000         0
         17   0.000000  0.000000  0.000000         0
         18   0.000000  0.000000  0.000000         0
         19   0.971575  0.950023  0.960678     14967
         20   0.695796  0.578923  0.632002      2173

avg / total   0.936612  0.902975  0.919009     17140

Classification report for turbine 25, turbine category 7.0
             precision    recall  f1-score   support

         10   0.000000  0.000000  0.000000         0
         11   0.000000  0.000000  0.000000         0
         12   0.000000  0.0000

Classification report for turbine 25, turbine category 9.0
             precision    recall  f1-score   support

         10   0.000000  0.000000  0.000000         0
         11   0.000000  0.000000  0.000000         0
         12   0.000000  0.000000  0.000000         0
         13   0.000000  0.000000  0.000000         0
         14   0.000000  0.000000  0.000000         0
         15   0.000000  0.000000  0.000000         0
         16   0.000000  0.000000  0.000000         0
         17   0.000000  0.000000  0.000000         0
         18   0.000000  0.000000  0.000000         0
         19   0.986750  0.962784  0.974620     15934
         20   0.715664  0.799337  0.755190      1206

avg / total   0.967676  0.951284  0.959180     17140

------------------------------------------------------------------------
Classification report for turbine 25, turbine category 10.0
             precision    recall  f1-score   support

         10   0.158301  0.218085  0.183445       188
         

Classification report for turbine 25, turbine category 16.0
             precision    recall  f1-score   support

         10   0.000000  0.000000  0.000000         0
         11   0.000000  0.000000  0.000000         0
         12   0.000000  0.000000  0.000000         0
         13   0.000000  0.000000  0.000000         0
         14   0.000000  0.000000  0.000000         0
         15   0.000000  0.000000  0.000000         0
         16   0.000000  0.000000  0.000000         0
         17   0.000000  0.000000  0.000000         0
         18   0.000000  0.000000  0.000000         0
         19   0.988471  0.850677  0.914412     16327
         20   0.342674  0.722017  0.464766       813

avg / total   0.957839  0.844574  0.893084     17140

Classification report for turbine 25, turbine category 16.0
             precision    recall  f1-score   support

         10   0.000000  0.000000  0.000000         4
         11   0.000000  0.000000  0.000000        33
         12   0.000000  0.00

Classification report for turbine 25, turbine category 19.0
             precision    recall  f1-score   support

         19   0.909132  0.959119  0.933457     14750
         20   0.618113  0.408368  0.491812      2390

avg / total   0.868552  0.882322  0.871874     17140

Classification report for turbine 25, turbine category 19.0
             precision    recall  f1-score   support

         19   0.971820  0.963119  0.967450     14967
         20   0.760728  0.807639  0.783482      2173

avg / total   0.945058  0.943407  0.944126     17140

Classification report for turbine 25, turbine category 19.0
             precision    recall  f1-score   support

         19   0.986546  0.975587  0.981036     15934
         20   0.718727  0.824212  0.767864      1206

avg / total   0.967702  0.964936  0.966037     17140

------------------------------------------------------------------------
Classification report for turbine 25, turbine category 20.0
             precision    recall  f1-score